In [1]:
import pandas as pd
import numpy as np
from notebooks.utils import read_file, tokenize_latex
df = read_file("notebooks/data/1601/*")
df["processed"] = df["text"].apply(lambda x: tokenize_latex(x))


error with: notebooks/data/1601/1601.06597.json


In [2]:
from gensim import corpora
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
equations = df["processed"].tolist()
dictionary = corpora.Dictionary(equations)

corpus = [dictionary.doc2bow(eq) for eq in equations]
corpora.MmCorpus.serialize('/tmp/equations.mm', corpus)
# our vector space model
print(corpus[1:5])

2017-11-08 23:09:09,770 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-11-08 23:09:10,318 : INFO : adding document #10000 to Dictionary(19919 unique tokens: [u'\\xi\\leq', u'4E+6F', u'0.000004', u'20\\%', u'i|S']...)
2017-11-08 23:09:10,880 : INFO : adding document #20000 to Dictionary(33619 unique tokens: [u'\\xi\\leq', u'eF', u'q-1', u'q-2', u'+i+1']...)
2017-11-08 23:09:11,408 : INFO : adding document #30000 to Dictionary(45883 unique tokens: [u'\\xi\\leq', u'eF', u'-5AB\\sqrt', u'JN2', u'q-1']...)
2017-11-08 23:09:11,951 : INFO : adding document #40000 to Dictionary(57458 unique tokens: [u'\\xi\\leq', u'eF', u'-5AB\\sqrt', u'JN2', u'q-1']...)
2017-11-08 23:09:12,509 : INFO : adding document #50000 to Dictionary(70025 unique tokens: [u'\\xi\\leq', u'eF', u'homomorphism', u'-5AB\\sqrt', u'JN2']...)
2017-11-08 23:09:12,881 : INFO : built Dictionary(76641 unique tokens: [u'\\xi\\leq', u'eF', u'homomorphism', u'-5AB\\sqrt', u'JN2']...) from 56379 documents (total 37

[[(2, 1), (4, 1), (5, 1), (6, 3), (8, 1), (11, 3), (14, 1), (15, 3), (16, 4), (17, 3), (21, 2), (23, 6), (25, 6), (26, 1), (27, 1), (28, 1)], [(2, 1), (3, 1), (4, 1), (5, 1), (6, 5), (7, 1), (8, 2), (11, 3), (14, 1), (15, 2), (16, 3), (17, 5), (18, 1), (19, 4), (21, 2), (23, 7), (24, 1), (25, 7), (27, 2), (29, 1), (30, 1), (31, 1)], [(1, 1), (2, 1), (4, 1), (5, 1), (8, 1), (10, 1), (14, 1), (16, 2), (17, 1), (21, 2), (23, 8), (25, 8), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1)], [(0, 1), (1, 1), (2, 1), (4, 3), (5, 3), (6, 1), (8, 1), (10, 1), (12, 3), (14, 1), (17, 2), (18, 2), (21, 2), (23, 8), (25, 8), (34, 1), (45, 1), (46, 1), (47, 2), (48, 2)]]


In [3]:
from gensim import corpora, models, similarities, matutils
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
from gensim import similarities
index = similarities.Similarity("notebooks/index/", corpus_tfidf, len(dictionary.keys()), num_best = 100)

2017-11-08 23:09:25,489 : INFO : collecting document frequencies
2017-11-08 23:09:25,492 : INFO : PROGRESS: processing document #0
2017-11-08 23:09:25,585 : INFO : PROGRESS: processing document #10000
2017-11-08 23:09:25,664 : INFO : PROGRESS: processing document #20000
2017-11-08 23:09:25,737 : INFO : PROGRESS: processing document #30000
2017-11-08 23:09:25,817 : INFO : PROGRESS: processing document #40000
2017-11-08 23:09:25,891 : INFO : PROGRESS: processing document #50000
2017-11-08 23:09:25,939 : INFO : calculating IDF weights for 56379 documents and 76640 features (1388440 matrix non-zeros)
2017-11-08 23:09:26,000 : INFO : starting similarity index under notebooks/index/
2017-11-08 23:09:30,399 : INFO : PROGRESS: fresh_shard size=10000
2017-11-08 23:09:34,455 : INFO : PROGRESS: fresh_shard size=20000
2017-11-08 23:09:39,026 : INFO : PROGRESS: fresh_shard size=30000
2017-11-08 23:09:40,362 : INFO : creating sparse index
2017-11-08 23:09:40,363 : INFO : creating sparse matrix from 

In [4]:
# looks like it works
sims = index[tfidf[corpus][1]] # perform a similarity query against the corpus
sims = sorted(sims, key=lambda item: -item[1])

# [df["text"].tolist()[i[0]] for i in sims]

2017-11-08 23:09:52,788 : INFO : creating sparse index
2017-11-08 23:09:52,790 : INFO : creating sparse matrix from corpus
2017-11-08 23:09:52,792 : INFO : PROGRESS: at document #0/23611
2017-11-08 23:09:53,582 : INFO : PROGRESS: at document #10000/23611
2017-11-08 23:09:54,374 : INFO : PROGRESS: at document #20000/23611
2017-11-08 23:09:54,694 : INFO : created <23611x76641 sparse matrix of type '<type 'numpy.float32'>'
	with 583829 stored elements in Compressed Sparse Row format>
2017-11-08 23:09:54,695 : INFO : creating sparse shard #1
2017-11-08 23:09:54,696 : INFO : saving index shard to notebooks/index/.1
2017-11-08 23:09:54,697 : INFO : saving SparseMatrixSimilarity object under notebooks/index/.1, separately None
2017-11-08 23:09:54,719 : INFO : saved notebooks/index/.1
2017-11-08 23:09:54,721 : INFO : loading SparseMatrixSimilarity object from notebooks/index/.1
2017-11-08 23:09:54,731 : INFO : loaded notebooks/index/.1


In [5]:
docs = ["".join(eq) for eq in df["processed"].tolist()]

In [6]:
import re
def clean_exp(exp):
    exp = re.sub("\\\\begin{equation[^}]*}","" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\end{equation[^}]*}","" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\begin{split[^}]*}","\\\\begin{aligned}" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\end{split[^}]*}","\\\\end{aligned}" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\begin{gather[^}]*}","\\\\begin{aligned}" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\end{gather[^}]*}","\\\\end{aligned}" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\begin{align[^}]*}","\\\\begin{aligned}" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\end{align[^}]*}","\\\\end{aligned}" ,exp, flags=re.IGNORECASE)
    exp = re.sub("\\\\label{[^}]*}","" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\n", "",exp, flags=re.IGNORECASE)
    exp = re.sub("\\$", "",exp , flags=re.IGNORECASE)
    return exp

clean_docs = map(clean_exp, docs)

In [7]:
from mathviz_hopper.src.table import Table 
f = lambda x: dictionary.doc2bow(tokenize_latex(x))
t = Table(index, f, clean_docs)
t.print_ipython()

8081


Bottle v0.13-dev server starting up (using MyWSGIRefServer())...
Listening on http://localhost:8081/
Hit Ctrl-C to quit.



listening
/Users/sam/Documents/fall17/mathviz/mathviz_hopper/src


In [8]:
import requests
r = requests.post('http://localhost:8081/query', json={"query": "\\begin{equation}\n\\Phi_{z}(L) = \\sum_{i=1}^{N} \\frac{1}{C_{i} \\times V_{\\rm max, i}} ,\n\\label{EQ1}\n\\end{equation}\n"})

127.0.0.1 - - [08/Nov/2017 23:10:01] "POST /query HTTP/1.1" 200 16258


In [9]:
r.json()

{u'neighbors': [{u'neighbor': u'\\Phi_{z}(L)=\\sum_{i=1}^{N}\\frac{1}{C_{i}\\timesV_{\\rmmax,i}},',
   u'similarity_score': 0.3113422393798828},
  {u'neighbor': u'S=\\sum^N_{i=1}S_i\\,\\,\\,,\\,\\,\\,L=\\sum^N_{i=1}L_i',
   u'similarity_score': 0.21985876560211182},
  {u'neighbor': u'q_{i}=\\int_{C^2_{i}}F^2,\\qquadi=0,1,',
   u'similarity_score': 0.21982091665267944},
  {u'neighbor': u'\\[\\Phi_{i}=\\{(\\tau_{i,1},\\xi_{i,1}),\\ldots,(\\tau_{i,N_T^i},\\xi_{i,N_T^i})\\}.\\]',
   u'similarity_score': 0.21745142340660095},
  {u'neighbor': u"\\sum_{i=1}^{n}a_i=\\sum_{i=1}^{n}a'_i,",
   u'similarity_score': 0.2167559564113617},
  {u'neighbor': u'\\xi^{(4)i}=(A^{i}{_{a}},\\beta^{i},e^{i}{_{a}},\\alpha^{i},\\lambda^{i}{_{a}},\\Gamma^{i},\\varphi,\\rho_{i},\\omega_{i},\\tau_{i},\\sigma),',
   u'similarity_score': 0.2106698751449585},
  {u'neighbor': u'R=\\frac{\\sum_i\\mathbf{r}_i}{N}.',
   u'similarity_score': 0.2102690488100052},
  {u'neighbor': u'u_{i,M}:=\\frac{1}{M_i}\\sum_{m=1}^{M_i}\\d

In [10]:
import requests
r = requests.get('http://localhost:8081/settings')

127.0.0.1 - - [08/Nov/2017 23:10:02] "GET /settings HTTP/1.1" 200 11827455


In [11]:
r.json()

127.0.0.1 - - [08/Nov/2017 23:10:07] "GET /settings HTTP/1.1" 200 11827455
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [12]:
from mathviz_hopper.src.test import Test 

t = Test()
t.print_html()

creating test instance


In [13]:

def insert(st, trie):
    i = 0
    for s in st:
        if s not in trie.keys(): 
            trie[s] = {}
        if i == 20:
            break
        trie = trie[s]
        i+=1
        
    if i == 20: 
        trie[st[i:]] = {}
        trie = trie[st[i:]]
    trie["full_word"] = 1
        
    
def construct_trie(list_of_str):
    trie = {}
    for st in list_of_str:
        insert(st, trie)
    return trie
    

In [14]:
docs_trie = construct_trie(clean_docs)


127.0.0.1 - - [08/Nov/2017 23:10:52] "GET /settings HTTP/1.1" 200 11827455


In [15]:

opts = {
    "columns": [
            {
                "Header": "Word",
                "accessor": "word"
            },
            {
                "Header": "Similarity",
                "accessor": "sim"
            }
   ],
    "port": "8081",
    "docs": docs_trie
}

In [16]:
import json
with open("mathviz_hopper/webpage/mathviz-js-components/public/settings.json", "w+") as f:
    json.dump(opts, f)

In [17]:
def word_exists(trie, word):
    
    for i, s in enumerate(word):        
        if s in trie.keys(): 
            trie = trie[s]
        else: return False
            
        if i == 19: 
            break
            
    if i == 19:
        s = word[i+1:]
        if s not in trie.keys(): return False
        trie = trie[s]

    return True if 'full_word' in trie.keys() else False
        
        

In [18]:
word_exists(trie, "fdakfkajshdfjkahsdfkjhadfdfdfdffdfdsdjkfhaskj")

NameError: name 'trie' is not defined

127.0.0.1 - - [08/Nov/2017 23:11:00] "POST /query HTTP/1.1" 200 23424
127.0.0.1 - - [08/Nov/2017 23:11:34] "POST /query HTTP/1.1" 200 28097


In [ ]:
docs = ["".join(eq) for eq in df["processed"].tolist()]
docs_trie = construct_trie(docs)